## Lib Imports

In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats

## Data Prep

In [2]:
def get_rookie_stats_season(season, per_mode):
    try:
        player_stats = leaguedashplayerstats.LeagueDashPlayerStats(
            season = season,
            season_type_all_star = 'Regular Season',
            player_experience_nullable = 'Rookie',
            per_mode_detailed = per_mode,
        ).get_data_frames()[0]

        player_stats['SEASON'] = season
    except Exception as e:
        print('Error: Could not get rookie stats for season', season)
        print(e)
        player_stats = pd.DataFrame()

    return player_stats

In [3]:
def format_season(season):
    return f'{season}-{str(season + 1)[-2:]}'

In [4]:
df_rookie_stats = pd.DataFrame()

for season in range(1950, 2025):
    season_str = format_season(season)
    rookie_stats = get_rookie_stats_season(season_str, 'PerGame')

    if rookie_stats.empty:
        print(f'No rookie stats found for {season_str} season')
        continue
    
    print(f'Found {rookie_stats.shape[0]} rookie stats for {season_str} season')
    df_rookie_stats = pd.concat([df_rookie_stats, rookie_stats])

No rookie stats found for 1950-51 season
No rookie stats found for 1951-52 season
No rookie stats found for 1952-53 season
No rookie stats found for 1953-54 season
No rookie stats found for 1954-55 season
No rookie stats found for 1955-56 season
No rookie stats found for 1956-57 season
No rookie stats found for 1957-58 season
No rookie stats found for 1958-59 season
No rookie stats found for 1959-60 season
No rookie stats found for 1960-61 season
No rookie stats found for 1961-62 season
No rookie stats found for 1962-63 season
No rookie stats found for 1963-64 season
No rookie stats found for 1964-65 season
No rookie stats found for 1965-66 season
No rookie stats found for 1966-67 season
No rookie stats found for 1967-68 season
No rookie stats found for 1968-69 season
No rookie stats found for 1969-70 season
No rookie stats found for 1970-71 season
No rookie stats found for 1971-72 season
No rookie stats found for 1972-73 season
No rookie stats found for 1973-74 season
No rookie stats 

In [5]:
df_rookie_stats.groupby('SEASON').size()

SEASON
1996-97     63
1997-98     64
1998-99     67
1999-00     63
2000-01     68
2001-02     69
2002-03     64
2003-04     67
2004-05     70
2005-06     85
2006-07     80
2007-08     64
2008-09     62
2009-10     57
2010-11     67
2011-12     81
2012-13     78
2013-14     76
2014-15     81
2015-16     73
2016-17     88
2017-18    116
2018-19    105
2019-20    118
2020-21     94
2021-22    126
2022-23     85
2023-24    101
dtype: int64

In [6]:
df_rookie_stats.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'SEASON'],
      dtype='object')

In [7]:
df_rookie_stats.drop(columns=['NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'WNBA_FANTASY_PTS_RANK', 'WNBA_FANTASY_PTS'], inplace=True)